In [1]:
import pandas as pd
import numpy as np
import requests 
import json
requests.get('https://api.github.com')


<Response [200]>

Importation des librairies et test de la libraire requests

In [2]:
account = pd.read_csv("account_groundTruth[92].csv")
account = account.iloc[:,[1,2]] #used because otherwise the account number are there twice 
# 0              0                       brotherlogic     bot        
#instead of
#0                       brotherlogic     bot
#print(account.to_string())
account.describe

<bound method NDFrame.describe of               login type_GT
0      brotherlogic     bot
1         coveralls     bot
2          renovate     bot
3          leocwlam     bot
4         fabric8cd     bot
...             ...     ...
1487       raphaelm   human
1488     TroyAlford   human
1489          daffl   human
1490  bryanyang0528   human
1491    ConnorRigby   human

[1492 rows x 2 columns]>

Utilisation de panda pour récupérer la base de données contenant les bots et compte humains.

In [3]:
values = account.values
checker = dict(values)
checker["brotherlogic"]

'bot'

Création d'un dictionnaire pour que lorsque la fonction de recherche pour les bots sera établie, si celle ci a bien détecté un bot ou non.

In [4]:
"""
def check(login,boolBot):
    if (boolBot):
        return (checker[login] == "bot")
    else:
        return (checker[login] == "human")
    
print(check("brotherlogic",True))
print(check("coveralls",False))
print(check("ConnorRigby",False))
print(check("ConnorRigby",True))
"""
def checkbot(login):
    return (checker[login] == "bot")

def checkhuman(login):
    return (checker[login] == "human")



Fonctions servant juste à vérifier correctement dans le dictionnaire

In [5]:
b_set = account.head(25) #bot set of size 25
h_set = account.tail(25) #human set of size 25
m_set = pd.concat([b_set,h_set])                 #mixed set of size 50
m_set = m_set.reset_index(drop = True)           #maybe not needed but "cleans" the dataframe
                                                 # so it goes from 1 to 50 instead of 1 to 25 then 1467 to 1491
#print(m_set.to_string())
#print(b_set.to_string())
#print(h_set.to_string())

Création d'une dataframe plus petite pour commencer

In [22]:
#r = requests.get('https://api.github.com/events')
r = requests.get("https://api.github.com/users/brotherlogic/events")
activity = r.json()
#print(activity)





In [23]:
def getactivities(list_login):
    for login in list_login:
        r = requests.get("https://api.github.com/users/{name}/events".format(name=login))
        activity = r.json()
        with open("./activities/{name}.json".format(name = login),"w") as outfile:
            json.dump(activity,outfile)

Création d'une fonction pour récupérer les activités des comptes.
Vérifier pourquoi certains fichiers json sont juste sous la forme root: "[]"
Vérifier aussi pourquoi les fichiers ne sont plus "reconnu" comme json (regarder fichier test.json et event.json)
Problème réglés (changer le r.text en r.json() )

In [24]:
sample = account.head(5)
login_sample = sample.login.values.tolist()
print(login_sample)

getactivities(login_sample)

['brotherlogic', 'coveralls', 'renovate', 'leocwlam', 'fabric8cd']


In [25]:
sample1 = account.tail(5)
login_sample1 = sample1.login.values.tolist()
print(login_sample1)

getactivities(login_sample1)

['raphaelm', 'TroyAlford', 'daffl', 'bryanyang0528', 'ConnorRigby']


Utilisation de la fonction et récupération une première fois des activités des comptes